<a href="https://kritikseth.github.io/ipynbtagredirect" target="_parent"><img src="https://raw.githack.com/kritikseth/kritikseth/master/assets/icons/kritik_ipynbtagredirect.svg" alt="Kritik Seth"/></a>

# EDA

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/kritikseth/Datasets/master/Analytics%20Vidhya/JanataHack%20Cross-sell%20Prediction/janatahack_cross_sell_prediction_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/kritikseth/Datasets/master/Analytics%20Vidhya/JanataHack%20Cross-sell%20Prediction/janatahack_cross_sell_prediction_test.csv')
ss = pd.read_csv('https://raw.githubusercontent.com/kritikseth/Datasets/master/Analytics%20Vidhya/JanataHack%20Cross-sell%20Prediction/janatahack_cross_sell_prediction_sample_submission.csv')

In [3]:
train.shape, test.shape

((381109, 12), (127037, 11))

In [4]:
(test.shape[0]/(train.shape[0]+test.shape[0]))*100

25.00009839691742

In [5]:
train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28,0,> 2 Years,Yes,40454,26,217,1
1,2,Male,76,1,3,0,1-2 Year,No,33536,26,183,0
2,3,Male,47,1,28,0,> 2 Years,Yes,38294,26,27,1
3,4,Male,21,1,11,1,< 1 Year,No,28619,152,203,0
4,5,Female,29,1,41,1,< 1 Year,No,27496,152,39,0


In [6]:
test.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,381110,Male,25,1,11.0,1,< 1 Year,No,35786.0,152.0,53
1,381111,Male,40,1,28.0,0,1-2 Year,Yes,33762.0,7.0,111
2,381112,Male,47,1,28.0,0,1-2 Year,Yes,40050.0,124.0,199
3,381113,Male,24,1,27.0,1,< 1 Year,Yes,37356.0,152.0,187
4,381114,Male,27,1,28.0,1,< 1 Year,No,59097.0,152.0,297


In [7]:
ss.head()

,id,Response
0,381110,0
1,381111,0
2,381112,0
3,381113,0
4,381114,0


In [8]:
train.drop(['id'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)

In [9]:
train.describe()

,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response
count,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000,381109.000000
mean,38.822584,0.997869,26.388807,0.458210,30564.389581,112.034295,154.347397,0.122563
std,15.511611,0.046110,13.229888,0.498251,17213.155057,54.203995,83.671304,0.327936
min,20.000000,0.000000,0.000000,0.000000,2630.000000,1.000000,10.000000,0.000000
25%,25.000000,1.000000,15.000000,0.000000,24405.000000,29.000000,82.000000,0.000000
50%,36.000000,1.000000,28.000000,0.000000,31669.000000,133.000000,154.000000,0.000000
75%,49.000000,1.000000,35.000000,1.000000,39400.000000,152.000000,227.000000,0.000000
max,85.000000,1.000000,52.000000,1.000000,540165.000000,163.000000,299.000000,1.000000


In [10]:
train['Annual_Premium'] = train['Annual_Premium'].apply(lambda x: np.log1p(x))
test['Annual_Premium'] = test['Annual_Premium'].apply(lambda x: np.log1p(x))

In [11]:
train['Gender'] = train['Gender'].astype(str)
train['Vehicle_Age'] = train['Vehicle_Age'].astype(str)
train['Vehicle_Damage'] = train['Vehicle_Damage'].astype(str)

test['Gender'] = test['Gender'].astype(str)
test['Vehicle_Age'] = test['Vehicle_Age'].astype(str)
test['Vehicle_Damage'] = test['Vehicle_Damage'].astype(str)

In [12]:
row_replace = {'> 2 Years':2, '1-2 Year':1, '< 1 Year':0}

train['Vehicle_Age'].replace(row_replace, inplace=True)
test['Vehicle_Age'].replace(row_replace, inplace=True)

In [13]:
le = LabelEncoder()

train['Gender'] = le.fit_transform(train['Gender'])
train['Vehicle_Damage'] = le.fit_transform(train['Vehicle_Damage'])

test['Gender'] = le.fit_transform(test['Gender'])
test['Vehicle_Damage'] = le.fit_transform(test['Vehicle_Damage'])

In [14]:
train['AmountVintage'] = (train['Annual_Premium']/365)*train['Vintage']
test['AmountVintage'] = (test['Annual_Premium']/365)*test['Vintage']

train['AgeVintage'] = train['Age']-(train['Vintage']/365)
test['AgeVintage'] = test['Age']-(test['Vintage']/365)

train_ageamt = ((train['Age']-25)*train['Annual_Premium']).copy()
test_ageamt  = ((test['Age']-25)*test['Annual_Premium']).copy()

train_ageamt[train_ageamt < 0] = 0
test_ageamt[test_ageamt < 0] = 0

train['AgeAmount'] = train_ageamt
test['AgeAmount'] = test_ageamt

In [15]:
train['Vintage'] = train['Vintage']/365
test['Vintage'] = test['Vintage']/365

In [16]:
train['Annual_Premium'] = train['Annual_Premium'].apply(lambda x: np.log1p(x))
train['AmountVintage'] = train['AmountVintage'].apply(lambda x: np.log1p(x))
train['AgeAmount'] = train['AgeAmount'].apply(lambda x: np.log1p(x))

test['Annual_Premium'] = test['Annual_Premium'].apply(lambda x: np.log1p(x))
test['AmountVintage'] = test['AmountVintage'].apply(lambda x: np.log1p(x))
test['AgeAmount'] = test['AgeAmount'].apply(lambda x: np.log1p(x))

In [17]:
train.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,AmountVintage,AgeVintage,AgeAmount
0,1,44,1,28,0,2,1,2.451690,26,0.594521,1,1.988784,43.405479,5.310992
1,1,76,1,3,0,1,0,2.435402,26,0.501370,0,1.828489,75.498630,6.277471
2,1,47,1,28,0,2,1,2.446952,26,0.073973,1,0.576972,46.926027,5.451758
3,1,21,1,11,1,0,0,2.421422,152,0.556164,0,1.903194,20.443836,0.000000
4,0,29,1,41,1,0,0,2.417861,152,0.106849,0,0.738214,28.893151,3.734983


In [18]:
train.drop(['AmountVintage', 'Driving_License', 'Vintage', 'Annual_Premium','Gender','Region_Code'], axis=1, inplace=True)
test.drop(['AmountVintage', 'Driving_License', 'Vintage', 'Annual_Premium','Gender','Region_Code'], axis=1, inplace=True)

In [19]:
X = train.drop(['Response'], axis=1)
Y = train['Response']

In [20]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [21]:
bf = SelectKBest(score_func=chi2, k='all')
bf.fit(X, Y)

SelectKBest(k='all', score_func=<function chi2 at 0x7f7e593d30d0>)

In [22]:
KBest = pd.DataFrame(list(zip(list(X.columns),list(bf.scores_))), columns=['Column Name', 'Score'])
KBest = KBest.sort_values(by=['Score'], ascending=False)
KBest = KBest.reset_index()
KBest.drop(['index'], axis=1, inplace=True)

In [23]:
KBest

,Column Name,Score
0,Policy_Sales_Channel,193218.436915
1,AgeVintage,29508.684753
2,Age,29179.114628
3,Previously_Insured,24033.827713
4,Vehicle_Damage,23700.035830
5,AgeAmount,22333.758648
6,Vehicle_Age,9909.317235


In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [25]:
X = scaler.fit_transform(X)

In [26]:
test = scaler.fit_transform(test)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

# Modelling

In [28]:
import lightgbm
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [29]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.decomposition import PCA

In [30]:
!pip install catboost

In [31]:
from catboost import CatBoostClassifier

# Modelling

In [35]:
# pca = PCA()
# pca.fit(X)

# Xpca = pca.transform(X)
# testpca = pca.transform(test)

In [32]:
catb = CatBoostClassifier()
catb= catb.fit(X, Y, cat_features=cat_col, 
               early_stopping_rounds=30, verbose=100)

In [33]:
proba = catb.predict_proba(test)[:,1]

In [34]:
xgb = XGBClassifier(colsample_bylevel=1.0,
                    colsample_bytree=0.8,
                    objective='binary:logistic',
                    gamma=0,
                    learning_rate=0.001,
                    max_depth=20,
                    min_child_weight=10,
                    n_estimators=1000,
                    reg_lambda=5,
                    metric='auc',
                    subsample=0.7,
                    n_jobs=-1,
                    seed=0)

lgb = LGBMClassifier(boosting_type='gbdt',
                    n_estimators=1000,
                    depth=100,
                    learning_rate=0.004,
                    objective='binary',
                    metric='auc',
                    is_unbalance=True,
                    colsample_bytree=0.9,
                    reg_lambda=2,
                    reg_alpha=2,
                    random_state=294,
                    n_jobs=-1)

hgb = HistGradientBoostingClassifier(max_iter=1500)

ada = AdaBoostClassifier(n_estimators=1000)

gbc = GradientBoostingClassifier(n_estimators=1000)

mlp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(7, 4), random_state=1)
sgd = SGDClassifier(loss="log", penalty="l2", max_iter=5)
dtc = DecisionTreeClassifier(random_state=0, max_depth=4, criterion='gini')

In [35]:
vcs = VotingClassifier(estimators=[('AdaBoost', ada),
                                   ('HistGB', hgb),
                                   ('GradB', gbc),
                                   ('XGBoost', xgb),
                                   ('LightGB', lgb)],
                        voting='soft')

In [36]:
vcs.fit(X, Y)

In [37]:
proba = vcs.predict_proba(test)[:,1]

In [38]:
prob = xgb.predict_proba(test)[:,1]
ss['Response'] = proba

In [39]:
ss.to_csv('ss.csv', index=False)

In [ ]:
xgb = XGBClassifier(colsample_bylevel=1.0,
                    colsample_bytree=0.8,
                    objective='binary:logistic',
                    gamma=0,
                    learning_rate=0.001,
                    max_depth=20,
                    min_child_weight=10,
                    n_estimators=1000,
                    reg_lambda=5,
                    metric='auc',
                    subsample=0.7,
                    n_jobs=-1,
                    seed=0)

lgb = LGBMClassifier(boosting_type='gbdt',
                    n_estimators=1000,
                    depth=100,
                    learning_rate=0.004,
                    objective='binary',
                    metric='auc',
                    is_unbalance=True,
                    colsample_bytree=0.9,
                    reg_lambda=2,
                    reg_alpha=2,
                    random_state=294,
                    n_jobs=-1)

hgb = HistGradientBoostingClassifier(max_iter=1500)

ada = AdaBoostClassifier(n_estimators=1000)

gbc = GradientBoostingClassifier(n_estimators=1000)

# mlp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(7, 4), random_state=1)
# sgd = SGDClassifier(loss="log", penalty="l2", max_iter=5)
# dtc = DecisionTreeClassifier(random_state=0, max_depth=4, criterion='gini')

sclf = StackingClassifier(classifiers=[xgb, lgb, hgb, gbc],
                          use_probas=True,
                          average_probas=False,
                          meta_classifier=ada)

In [ ]:
sclf.fit(X, Y)

In [ ]:
proba = sclf.predict_proba(test)[:,1]

In [ ]:
ss['Response'] = proba
ss.to_csv('ss.csv', index=False)